In [1]:
!pip install pickle

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [1]:
#Libreria
import pickle
import spacy
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
!python -m spacy download en_core_web_md

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


2022-03-30 20:03:37.535781: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll


In [3]:
#Carga del modelo
nlp = spacy.load("en_core_web_md")

In [4]:
df = pd.read_csv('cannabis.csv')
df.head()

,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


In [5]:
def list_effects(data=df.Effects):
    """Funcion para agregar todos los efectos únicos asociados con la columna Efectos del DataFrame.
    
    Args:
        data (pandas.Series): DataFrame volcado en una columna. Default 'Effects'
    Returns:
        effects_list (list): lista de todos los efectos
    """
    effects_list = []
    
    # Divide cada lista de efectos y agregue a cada uno, lowered effect a la lista
    for i in range(0, len(df)):
        effects = data[i].split(",")
        for effect in effects:
            effects_list.append(effect.lower())
    
    return set(effects_list)

In [6]:
# Lista unica de efectos
all_effects = list_effects()
all_effects

{'aroused',
 'creative',
 'dry',
 'energetic',
 'euphoric',
 'focused',
 'giggly',
 'happy',
 'hungry',
 'mouth',
 'none',
 'relaxed',
 'sleepy',
 'talkative',
 'tingly',
 'uplifted'}

In [7]:
# Eliminando efectos innecesarios como "dry", "none" ect
bad_effects = ['dry', 'mouth', 'aroused', 'none']
for effect in bad_effects:
    all_effects.remove(effect)

In [8]:
# Usando capitalize para presentaciones, como ser las mayusculas
effects_list = []

for effect in all_effects:
    effects_list.append(effect.capitalize())

In [9]:
# Elminando valores NaN
df = df[~df['Description'].isnull()].reset_index()

In [10]:
# Eliminando 'none' de las descripciones
df = df[~(df.Description == 'None')].reset_index()

In [11]:
df = df.drop(columns=['level_0', 'index'])
df

,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."
...,...,...,...,...,...,...
2309,Zeus-Og,hybrid,4.7,"Happy,Uplifted,Relaxed,Euphoric,Energetic","Earthy,Woody,Pine",Zeus OG is a hybrid cross between Pineapple OG...
2310,Zkittlez,indica,4.6,"Relaxed,Happy,Euphoric,Uplifted,Sleepy","Sweet,Berry,Grape",Zkittlez is an indica-dominant mix of Grape Ap...
2311,Zombie-Kush,indica,5.0,"Relaxed,Sleepy,Talkative,Euphoric,Happy","Earthy,Sweet,Spicy/Herbal",Zombie Kush by Ripper Seeds comes from two dif...
2312,Zombie-Og,indica,4.4,"Relaxed,Sleepy,Euphoric,Happy,Hungry","Sweet,Earthy,Pungent",If you’re looking to transform into a flesh-ea...


In [12]:
#Nuevo csv
df.to_csv('new_cannabis.csv')

In [13]:
def preprocessor(doc):
    """Procesa previamente los datos de entrada de texto utilizando la funcionalidad spaCy.

    Args:
        doc (list): Lista de datos de entrada a procesar (input)
    Returns:
        new_text (str): Nuevo documento
    """
    doc = nlp(doc)
    new_text = " ".join([token.lemma_.lower() for token in doc if not
                         token.is_stop and not token.is_punct])

    return new_text

# Aplicando preproceso a la data y guardandola 
df['Processed'] = df['Description'].apply(preprocessor)

In [14]:
# Inicio de la vectorizacion y el entrenamiento
vect = TfidfVectorizer(ngram_range=(1, 2))
vect.fit(df['Processed'])

# Transformando data para el entrenamiento de NN
dtm = vect.transform(df['Processed'])

In [15]:
# Iniciacion y entramiento del modelo
nn = NearestNeighbors(algorithm='kd_tree', n_neighbors=50, n_jobs=-1)
nn.fit(dtm)

D:\Anaconda\lib\site-packages\sklearn\neighbors\_base.py:415: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


NearestNeighbors(algorithm='kd_tree', n_jobs=-1, n_neighbors=50)

In [20]:
# Descripcion para prueba, en "my_desc" ingresar la solicitud
my_desc = "pain"

# Preprocessor y vectorizacion de la descripcion
my_desc = preprocessor(my_desc)
my_desc_vect = vect.transform([my_desc])

In [21]:
# Encontrando el NN mas cercano o descripcion similar
dist, ind = nn.kneighbors(my_desc_vect)

In [22]:
dist

array([[1.33604718, 1.34087318, 1.35224823, 1.35445014, 1.35474008,
        1.3556184 , 1.35571337, 1.35766905, 1.35954969, 1.36083779,
        1.36495062, 1.36534756, 1.36540802, 1.36631221, 1.36670951,
        1.36684907, 1.36764457, 1.36822016, 1.36867024, 1.37107286,
        1.3719927 , 1.37216712, 1.37243375, 1.37383237, 1.37385166,
        1.37446092, 1.3750177 , 1.37509262, 1.37519092, 1.37534741,
        1.37537455, 1.37551324, 1.37555441, 1.37567992, 1.37579171,
        1.37595652, 1.37604935, 1.37614468, 1.3762098 , 1.37631166,
        1.37639882, 1.37676328, 1.37683483, 1.37700085, 1.37701189,
        1.3770711 , 1.37718061, 1.37719601, 1.37745845, 1.37769068]])

In [23]:
ind

array([[1903, 1638,   19,  935,  396,  927, 1272, 1970, 1895, 1534, 1747,
         807, 2234, 1634, 1053, 1556, 1954, 1081, 1363,  996, 1383, 1134,
         662, 1295, 1481, 1216, 1108,  256, 1199,  346,  826,  669,  410,
        1501,  472,  172,  708,  268,  764,  219,  619, 1142, 2160,  682,
         340, 1688,  960,  354,  125, 1068]], dtype=int64)

In [24]:
df["Description"][3]

'13 Dawgs is a hybrid of G13 and Chemdawg genetics bred by Canadian LP Delta 9 BioTech. The two potent strains mix to create a balance between indica and sativa effects. 13 Dawgs has a sweet earthy musk that brings a blend of woody\xa0citrus flavors. The effects of 13 Dawgs induce a happy, relaxed body buzz with a creative and focused mind that counters depression and stimulates the appetite.'

In [28]:
## Pickle objects
pickle.dump(effects_list, open('effects_list.pkl', 'wb'))
pickle.dump(vect, open('vectorizer.pkl', 'wb'))
pickle.dump(nn, open('model.pkl', 'wb'))